In [1]:
from datetime import datetime, timedelta
import pandas as pd
from sqlalchemy import create_engine

# Database connection setup
db_config = {
    'host': '10.10.10.71',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'kaviyam123',
    'port': '5432'
}
connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)

# Step 1: Load data
query = "SELECT * FROM backup_overall_merged_data_31_12;"
df = pd.read_sql(query, con=engine)

In [2]:
# Convert dates to datetime
df['Policy Start Date'] = pd.to_datetime(df['Policy Start Date'], errors='coerce')
df['Policy End Date'] = pd.to_datetime(df['Policy End Date'], errors='coerce')

# Step 2: Remove invalid rows in 'Total Premium Payable'
df = df[~df['Total Premium Payable '].isnull()]  # Remove null values
df = df[~df['Total Premium Payable '].astype(str).str.isalpha()]  # Remove categorical values
df = df[df['Total Premium Payable '] != 0]  # Remove rows with 0

In [ ]:
# Ensure the 'booked' column is explicitly cast to a string type
if 'booked' not in df.columns or df['booked'].isnull().all():
    df['booked'] = None  # If the column doesn't exist or is completely null, initialize it
df['booked'] = df['booked'].astype('str')  # Convert to string to handle assignments of '0', '1', '-'

today = pd.Timestamp.now().normalize()

def update_booked(group):
    # Sort policies by start date within the group
    group = group.sort_values(by='Policy Start Date')
    
    for i in range(len(group) - 1):
        current_policy = group.iloc[i]
        next_policy = group.iloc[i + 1]
        
        # Only process rows where booked is NULL or 'None'
        if pd.isnull(current_policy['booked']) or current_policy['booked'] == 'None':
            # Check if the next policy starts the day after the current one ends
            if next_policy['Policy Start Date'] == current_policy['Policy End Date'] + timedelta(days=1):
                if next_policy['Policy Start Date'] > today:
                    group.loc[current_policy.name, 'booked'] = '1'
                else:
                    group.loc[current_policy.name, 'booked'] = '1'
            elif next_policy['Policy Start Date'] > current_policy['Policy End Date'] + timedelta(days=1):
                group.loc[current_policy.name, 'booked'] = '1'
    
    # Handle the last policy for the customer
    last_policy = group.iloc[-1]
    if pd.isnull(last_policy['booked']) or last_policy['booked'] == 'None':
        if last_policy['Policy End Date'] >= today:
            group.loc[last_policy.name, 'booked'] = '-'
        else:
            group.loc[last_policy.name, 'booked'] = '0'
    
    return group

# Apply the function to each group of policies
df = df.groupby('Policy No').apply(update_booked).reset_index(drop=True)

In [ ]:
# Step 4: Handle duplicates and prioritize
duplicates = df[df.duplicated(subset=['Policy No', 'Policy Start Date', 'Policy End Date'], keep=False)]

def prioritize_rows(group):
    group = group.assign(null_count=group.isnull().sum(axis=1))
    group = group.sort_values(by=['null_count', 'booked'], ascending=[True, False])
    return group.iloc[0]

cleaned_duplicates = (
    duplicates.groupby(['Policy No', 'Policy Start Date', 'Policy End Date'])
    .apply(prioritize_rows)
    .reset_index(drop=True)
)

df_cleaned = pd.concat([df, cleaned_duplicates]).drop_duplicates(subset=['Policy No', 'Policy Start Date', 'Policy End Date'], keep='last')

In [ ]:
def correct_booked(group):
    group = group.sort_values(by='Policy Start Date')
    for i in range(len(group) - 1):
        current_policy = group.iloc[i]
        next_policy = group.iloc[i + 1]
        
        # Check if the current policy is marked as '0' but satisfies the condition
        if current_policy['booked'] == '0':
            # Check if the next policy starts the day after the current one ends
            if next_policy['Policy Start Date'] == current_policy['Policy End Date'] + timedelta(days=1):
                if next_policy['Policy Start Date'] > today:
                    group.loc[current_policy.name, 'booked'] = '1'
                else:
                    group.loc[current_policy.name, 'booked'] = '1'
            # Check if the next policy starts after a gap
            elif next_policy['Policy Start Date'] > current_policy['Policy End Date'] + timedelta(days=1):
                if next_policy['Policy Start Date'] > today:
                    group.loc[current_policy.name, 'booked'] = '1'
    
    return group

# Apply the correction to each group of policies
df_cleaned = df_cleaned.groupby('Policy No').apply(correct_booked).reset_index(drop=True)

# Save the cleaned dataset back to the database
df_cleaned.to_sql('cleaned_overall_merged_base_pr_data', con=engine, if_exists='replace', index=False)

In [1]:
from datetime import datetime, timedelta
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.sql import text

# Database connection setup
db_config = {
    'host': '10.10.10.71',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'kaviyam123',
    'port': '5432'
}
connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)

# Step 1: Load data
query = "SELECT * FROM backup_overall_merged_data_31_12;"
df = pd.read_sql(query, con=engine)

# Convert dates to datetime
df['Policy Start Date'] = pd.to_datetime(df['Policy Start Date'], errors='coerce')
df['Policy End Date'] = pd.to_datetime(df['Policy End Date'], errors='coerce')

# Step 2: Remove invalid rows in 'Total Premium Payable'
df['Total Premium Payable '] = pd.to_numeric(df['Total Premium Payable '].astype(str).str.strip(), errors='coerce')
df = df[df['Total Premium Payable '].notnull() & (df['Total Premium Payable '] != 0)]

# Initialize 'booked' column
if 'booked' not in df.columns:
    df['booked'] = None
df['booked'] = df['booked'].fillna('').astype(str)

today = pd.Timestamp.now().normalize()

def update_booked(group):
    # Sort policies by start date within the group
    group = group.sort_values(by='Policy Start Date')

    for i in range(len(group) - 1):
        current_policy = group.iloc[i]
        next_policy = group.iloc[i + 1]

        # Only process rows where booked is empty
        if current_policy['booked'] in ['', 'None']:
            # Check if the next policy starts the day after the current one ends
            if next_policy['Policy Start Date'] == current_policy['Policy End Date'] + timedelta(days=1):
                group.loc[current_policy.name, 'booked'] = '1'
            elif next_policy['Policy Start Date'] > current_policy['Policy End Date'] + timedelta(days=1):
                group.loc[current_policy.name, 'booked'] = '1'

    # Handle the last policy for the customer
    last_policy = group.iloc[-1]
    if last_policy['booked'] in ['', 'None']:
        if last_policy['Policy End Date'] >= today:
            group.loc[last_policy.name, 'booked'] = '-'
        else:
            group.loc[last_policy.name, 'booked'] = '0'

    return group

# Apply the function to each group of policies
df = df.groupby('Policy No').apply(update_booked).reset_index(drop=True)

# Step 4: Handle duplicates and prioritize
def prioritize_rows(group):
    group = group.assign(null_count=group.isnull().sum(axis=1))
    group = group.sort_values(by=['null_count', 'booked', 'Policy Start Date'], ascending=[True, False, True])
    return group.iloc[0]

duplicates = df[df.duplicated(subset=['Policy No', 'Policy Start Date', 'Policy End Date'], keep=False)]
cleaned_duplicates = (
    duplicates.groupby(['Policy No', 'Policy Start Date', 'Policy End Date'])
    .apply(prioritize_rows)
    .reset_index(drop=True)
)

df_cleaned = df.drop_duplicates(subset=['Policy No', 'Policy Start Date', 'Policy End Date'], keep=False)
df_cleaned = pd.concat([df_cleaned, cleaned_duplicates], ignore_index=True)

def correct_booked(group):
    group = group.sort_values(by='Policy Start Date')
    for i in range(len(group) - 1):
        current_policy = group.iloc[i]
        next_policy = group.iloc[i + 1]

        # Check if the current policy is marked as '0' but satisfies the condition
        if current_policy['booked'] == '0':
            if next_policy['Policy Start Date'] == current_policy['Policy End Date'] + timedelta(days=1):
                group.loc[current_policy.name, 'booked'] = '1'
            elif next_policy['Policy Start Date'] > current_policy['Policy End Date'] + timedelta(days=1):
                group.loc[current_policy.name, 'booked'] = '1'

    return group

# Apply the correction to each group of policies
df_cleaned = df_cleaned.groupby('Policy No').apply(correct_booked).reset_index(drop=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_10204\254601383.py:63: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Policy No').apply(update_booked).reset_index(drop=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_10204\254601383.py:74: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(prioritize_rows)
C:\Users\Admin\AppData\Local\Temp\ipykernel_10204\254601383.py:97: DeprecationWarning: 

In [2]:
df_cleaned.to_sql(
    'cleaned_overall_merged_base_pr_data',
    con=engine,
    if_exists='replace',
    index=False,
    chunksize=100000  # Adjust chunk size based on performance
)

1498

In [ ]:
# Save the cleaned dataset back to the database
with engine.connect() as connection:
    connection.execute(text("DROP TABLE IF EXISTS cleaned_overall_merged_base_pr_data;"))
    df_cleaned.to_sql('cleaned_overall_merged_base_pr_data', con=engine, if_exists='replace', index=False)